In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 119 kB in 2s (76.1 kB/s)
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
df.createOrReplaceTempView("home_sales")
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = '''
SELECT ROUND(AVG(price), 2), YEAR(date_format(date, 'yyyy-MM-dd')), AVG(bedrooms)
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date_format(date, 'yyyy-MM-dd'))
ORDER BY YEAR(date_format(date, 'yyyy-MM-dd')) DESC
'''
result = spark.sql(query)

result.show()

+--------------------+-----------------------------------+-------------+
|round(avg(price), 2)|year(date_format(date, yyyy-MM-dd))|avg(bedrooms)|
+--------------------+-----------------------------------+-------------+
|           296363.88|                               2022|          4.0|
|           301819.44|                               2021|          4.0|
|           298353.78|                               2020|          4.0|
|            300263.7|                               2019|          4.0|
+--------------------+-----------------------------------+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = '''
SELECT ROUND(AVG(price), 2), YEAR(date_format(date, 'yyyy-MM-dd')), AVG(bedrooms), AVG(bathrooms)
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY YEAR(date_format(date, 'yyyy-MM-dd'))
ORDER BY YEAR(date_format(date, 'yyyy-MM-dd')) DESC
'''
result = spark.sql(query)

result.show()


+--------------------+-----------------------------------+-------------+--------------+
|round(avg(price), 2)|year(date_format(date, yyyy-MM-dd))|avg(bedrooms)|avg(bathrooms)|
+--------------------+-----------------------------------+-------------+--------------+
|           292725.69|                               2022|          3.0|           3.0|
|           294211.46|                               2021|          3.0|           3.0|
|           294204.16|                               2020|          3.0|           3.0|
|           287287.82|                               2019|          3.0|           3.0|
+--------------------+-----------------------------------+-------------+--------------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = '''
SELECT ROUND(AVG(price), 2), YEAR(date_format(date, 'yyyy-MM-dd')),
AVG(bedrooms),
MIN(sqft_living),
ROUND(AVG(sqft_living), 2)

FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY YEAR(date_format(date, 'yyyy-MM-dd'))
ORDER BY YEAR(date_format(date, 'yyyy-MM-dd')) DESC
'''
result = spark.sql(query)

result.show()


+--------------------+-----------------------------------+-------------+----------------+--------------------------+
|round(avg(price), 2)|year(date_format(date, yyyy-MM-dd))|avg(bedrooms)|min(sqft_living)|round(avg(sqft_living), 2)|
+--------------------+-----------------------------------+-------------+----------------+--------------------------+
|           290242.99|                               2022|          3.0|            2000|                   2267.94|
|           296330.96|                               2021|          3.0|            2001|                   2281.59|
|           292289.09|                               2020|          3.0|            2001|                   2313.66|
|           289859.14|                               2019|          3.0|            2000|                   2275.18|
+--------------------+-----------------------------------+-------------+----------------+--------------------------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = '''
SELECT view, ROUND(AVG(price), 2), YEAR(date_format(date, 'yyyy-MM-dd')),
AVG(bedrooms),
MIN(sqft_living),
ROUND(AVG(sqft_living), 2)

FROM home_sales
GROUP BY YEAR(date_format(date, 'yyyy-MM-dd')), view
HAVING AVG(price) >= 350000
ORDER BY view DESC
'''
result = spark.sql(query)

result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+-----------------------------------+-----------------+----------------+--------------------------+
|view|round(avg(price), 2)|year(date_format(date, yyyy-MM-dd))|    avg(bedrooms)|min(sqft_living)|round(avg(sqft_living), 2)|
+----+--------------------+-----------------------------------+-----------------+----------------+--------------------------+
| 100|            977881.5|                               2019|              5.8|            3146|                    4686.8|
| 100|            533862.0|                               2022|              4.0|            3773|                    3773.0|
| 100|           1147457.5|                               2020|              6.1|            3944|                    5194.3|
| 100|           994213.43|                               2021|5.571428571428571|            3425|                   4569.29|
|  99|          1134806.56|                               2019|5.333333333333333|            4105|                   5

In [9]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()


DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

cached_result = spark.sql(query)
cached_result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+-----------------------------------+-----------------+----------------+--------------------------+
|view|round(avg(price), 2)|year(date_format(date, yyyy-MM-dd))|    avg(bedrooms)|min(sqft_living)|round(avg(sqft_living), 2)|
+----+--------------------+-----------------------------------+-----------------+----------------+--------------------------+
| 100|            977881.5|                               2019|              5.8|            3146|                    4686.8|
| 100|            533862.0|                               2022|              4.0|            3773|                    3773.0|
| 100|           1147457.5|                               2020|              6.1|            3944|                    5194.3|
| 100|           994213.43|                               2021|5.571428571428571|            3425|                   4569.29|
|  99|          1134806.56|                               2019|5.333333333333333|            4105|                   5

In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("parquet")

In [13]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('parquet')


In [14]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_df_home_sales')


In [15]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
query = '''
SELECT view, ROUND(AVG(price), 2), YEAR(date_format(date, 'yyyy-MM-dd')),
AVG(bedrooms),
MIN(sqft_living),
ROUND(AVG(sqft_living), 2)

FROM p_df_home_sales
GROUP BY YEAR(date_format(date, 'yyyy-MM-dd')), view
HAVING AVG(price) >= 350000
ORDER BY view DESC
'''

spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+-----------------------------------+-----------------+----------------+--------------------------+
|view|round(avg(price), 2)|year(date_format(date, yyyy-MM-dd))|    avg(bedrooms)|min(sqft_living)|round(avg(sqft_living), 2)|
+----+--------------------+-----------------------------------+-----------------+----------------+--------------------------+
| 100|            977881.5|                               2019|              5.8|            3146|                    4686.8|
| 100|            533862.0|                               2022|              4.0|            3773|                    3773.0|
| 100|           1147457.5|                               2020|              6.1|            3944|                    5194.3|
| 100|           994213.43|                               2021|5.571428571428571|            3425|                   4569.29|
|  99|          1134806.56|                               2019|5.333333333333333|            4105|                   5

In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")


DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False